<a href="https://colab.research.google.com/github/stephenbiu/LLMsStudy/blob/master/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SimplerEnv: Simulated Manipulation Policy Evaluation for Real-World Robots

- Project page: <https://simpler-env.github.io/>
- Code: <https://github.com/simpler-env/SimplerEnv>

## Installation


In [1]:
#@title [!Important]Please use a GPU runtime.
!nvidia-smi

Thu Dec 12 03:08:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# @title Install vulkan for rendering
!apt-get install -yqq --no-install-recommends libvulkan-dev vulkan-tools
# below fixes some bugs introduced by some recent Colab changes
!mkdir -p /usr/share/vulkan/icd.d
!wget -q -P /usr/share/vulkan/icd.d https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/nvidia_icd.json
!wget -q -O /usr/share/glvnd/egl_vendor.d/10_nvidia.json https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/10_nvidia.json

Selecting previously unselected package libvulkan1:amd64.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../libvulkan1_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan1:amd64 (1.3.204.1-2) ...
Selecting previously unselected package libvulkan-dev:amd64.
Preparing to unpack .../libvulkan-dev_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan-dev:amd64 (1.3.204.1-2) ...
Selecting previously unselected package vulkan-tools.
Preparing to unpack .../vulkan-tools_1.3.204.0+dfsg1-1_amd64.deb ...
Unpacking vulkan-tools (1.3.204.0+dfsg1-1) ...
Setting up libvulkan1:amd64 (1.3.204.1-2) ...
Setting up libvulkan-dev:amd64 (1.3.204.1-2) ...
Setting up vulkan-tools (1.3.204.0+dfsg1-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbo

In [3]:
# @title Make sure vulkan is installed correctly
!vulkaninfo | head -n 5

'DISPLAY' environment variable not set... skipping surface info
error: XDG_RUNTIME_DIR not set in the environment.
VULKANINFO

Vulkan Instance Version: 1.3.204


In [4]:
# @title Install Real2Sim
!pip install numpy==1.24.4
!pip install orbax-checkpoint==0.4.4
!pip install scipy==1.12.0
!pip install keras==2.15.0
!pip install tensorflow==2.15.1
!git clone https://github.com/simpler-env/ManiSkill2_real2sim.git
!pip install -e ./ManiSkill2_real2sim
!git clone https://github.com/simpler-env/SimplerEnv.git
!pip install -e ./SimplerEnv
!mkdir ./SimplerEnv/checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: orbax-checkpoint
    Found existing installation: orbax-checkpoint 0.6.4
    Uninstalling orbax-checkpoint-0.6.4:
      Successfully uninstalled orbax-checkpoint-0.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 

In [5]:
#@title [Optional]Install RT-1 dependencies
! pip install --quiet tf_agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 34.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.36 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
langchain-core 0.3.21 requires typing-extensions>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
nibabel 5.3.2 requires typing-extensions>=4.6; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
openai 1.54.5 

In [6]:
#@title [Optional]Install Octo dependencies
!git clone https://github.com/octo-models/octo
!cd ./octo && git checkout 653c54acde686fde619855f2eac0dd6edad7116b && cd ..
!pip install -e ./octo
!pip install --upgrade "jax[cuda12_pip]==0.4.20" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html --no-deps
!pip install --upgrade "jaxlib[cuda12_pip]==0.4.20" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html --no-deps
!pip install jaxlib==0.4.20+cuda12.cudnn89
!pip install distrax==0.1.5 "einops>= 0.6.1"

Cloning into 'octo'...
remote: Enumerating objects: 7382, done.
remote: Counting objects: 100% (1066/1066), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 7382 (delta 983), reused 939 (delta 939), pack-reused 6316 (from 1)
Receiving objects: 100% (7382/7382), 23.75 MiB | 15.46 MiB/s, done.
Resolving deltas: 100% (4768/4768), done.
Note: switching to '653c54acde686fde619855f2eac0dd6edad7116b'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 653c54a Merge pull request #31 from octo-models/

In [7]:
# @title Install other requirements
!pip install --quiet mediapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.7 MB/s eta 0:00:00


In [8]:
# @title [Important]Post Installation

# run this so local pip installs are recognized
import site
site.main()

## Create a Simulated Environment and Take Random Actions

In [9]:
import simpler_env
from simpler_env.utils.env.observation_utils import get_image_from_maniskill2_obs_dict
import mediapy
import sapien.core as sapien

task_name = "google_robot_pick_coke_can"  # @param ["google_robot_pick_coke_can", "google_robot_move_near", "google_robot_open_drawer", "google_robot_close_drawer", "widowx_spoon_on_towel", "widowx_carrot_on_plate", "widowx_stack_cube", "widowx_put_eggplant_in_basket"]

if 'env' in locals():
  print("Closing existing env")
  env.close()
  del env
env = simpler_env.make(task_name)
# Colab GPU does not supoort denoiser
sapien.render_config.rt_use_denoiser = False
obs, reset_info = env.reset()
instruction = env.get_language_instruction()
print("Reset info", reset_info)
print("Instruction", instruction)

frames = []
done, truncated = False, False
while not (done or truncated):
   # action[:3]: delta xyz; action[3:6]: delta rotation in axis-angle representation;
   # action[6:7]: gripper (the meaning of open / close depends on robot URDF)
   image = get_image_from_maniskill2_obs_dict(env, obs)
   action = env.action_space.sample() # replace this with your policy inference
   obs, reward, done, truncated, info = env.step(action)
   frames.append(image)

episode_stats = info.get('episode_stats', {})
print("Episode stats", episode_stats)
mediapy.show_video(frames, fps=10)

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_language_instruction to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_language_instruction` for environment variables or `env.get_wrapper_attr('get_language_instruction')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.robot_uid to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.robot_uid` for environment variables or `env.get_wrapper_attr('robot_uid')` that will search the reminding wrappers.
  logger.warn(


Reset info {'scene_name': 'google_pick_coke_can_1_v4', 'scene_offset': None, 'scene_pose': None, 'scene_table_height': 0.87, 'urdf_version': 'recolor_tabletop_visual_matching_1', 'rgb_overlay_path': '/content/ManiSkill2_real2sim/data/real_inpainting/google_coke_can_real_eval_1.png', 'rgb_overlay_cameras': ['overhead_camera'], 'rgb_overlay_mode': 'background', 'disable_bad_material': False, 'model_id': 'opened_coke_can', 'model_scale': 1.0, 'distractor_model_ids': None, 'distractor_model_scales': None, 'obj_init_pose_wrt_robot_base': Pose([0.587925, -0.0238302, 0.840576], [0.707052, -0.0081018, -0.01162, -0.70702]), 'orientation': 'laid_vertically'}
Instruction pick coke can
Episode stats OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', True)])


This browser does not support the video tag.


## Run Inference on Simulated Environments

In [10]:
# @title Setup

import os
import numpy as np
import simpler_env
from simpler_env.utils.env.observation_utils import get_image_from_maniskill2_obs_dict
import mediapy


RT_1_CHECKPOINTS = {
    "rt_1_x": "rt_1_x_tf_trained_for_002272480_step",
    "rt_1_400k": "rt_1_tf_trained_for_000400120",
    "rt_1_58k": "rt_1_tf_trained_for_000058240",
    "rt_1_1k": "rt_1_tf_trained_for_000001120",
}


def get_rt_1_checkpoint(name, ckpt_dir="./SimplerEnv/checkpoints"):
  assert name in RT_1_CHECKPOINTS, name
  ckpt_name = RT_1_CHECKPOINTS[name]
  ckpt_path = os.path.join(ckpt_dir, ckpt_name)
  if not os.path.exists(ckpt_path):
    if name == "rt_1_x":
      !gsutil -m cp -r gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/{ckpt_name}.zip {ckpt_dir}
      !unzip {ckpt_dir}/{ckpt_name}.zip -d {ckpt_dir}
    else:
      !gsutil -m cp -r gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/{ckpt_name} {ckpt_dir}
  return ckpt_path

In [11]:
# @title Select your model and environment

task_name = "google_robot_pick_coke_can"  # @param ["google_robot_pick_coke_can", "google_robot_move_near", "google_robot_open_drawer", "google_robot_close_drawer", "widowx_spoon_on_towel", "widowx_carrot_on_plate", "widowx_stack_cube", "widowx_put_eggplant_in_basket"]

if 'env' in locals():
  print("Closing existing env")
  env.close()
  del env
env = simpler_env.make(task_name)

# Note: we turned off the denoiser as the colab kernel will crash if it's turned on
# To use the denoiser, please git clone our SIMPLER environments
# and perform evaluations locally.
sapien.render_config.rt_use_denoiser = False

obs, reset_info = env.reset()
instruction = env.get_language_instruction()
print("Reset info", reset_info)
print("Instruction", instruction)

if "google" in task_name:
  policy_setup = "google_robot"
else:
  policy_setup = "widowx_bridge"

Closing existing env
Reset info {'scene_name': 'google_pick_coke_can_1_v4', 'scene_offset': None, 'scene_pose': None, 'scene_table_height': 0.87, 'urdf_version': 'recolor_tabletop_visual_matching_1', 'rgb_overlay_path': '/content/ManiSkill2_real2sim/data/real_inpainting/google_coke_can_real_eval_1.png', 'rgb_overlay_cameras': ['overhead_camera'], 'rgb_overlay_mode': 'background', 'disable_bad_material': False, 'model_id': 'opened_coke_can', 'model_scale': 1.0, 'distractor_model_ids': None, 'distractor_model_scales': None, 'obj_init_pose_wrt_robot_base': Pose([0.587925, -0.0238302, 0.840576], [0.707052, -0.0081018, -0.01162, -0.70702]), 'orientation': 'laid_vertically'}
Instruction pick coke can


In [12]:
# @title Select your model and environment
model_name = "rt_1_x" # @param ["rt_1_x", "rt_1_400k", "rt_1_58k", "rt_1_1k", "octo-base", "octo-small"]

if "rt_1" in model_name:
  from simpler_env.policies.rt1.rt1_model import RT1Inference

  ckpt_path = get_rt_1_checkpoint(model_name)
  model = RT1Inference(saved_model_path=ckpt_path, policy_setup=policy_setup)
elif "octo" in model_name:
  from simpler_env.policies.octo.octo_model import OctoInference

  model = OctoInference(model_type=model_name, policy_setup=policy_setup, init_rng=0)
else:
  raise ValueError(model_name)


Copying gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/rt_1_x_tf_trained_for_002272480_step.zip...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1/1 files][738.7 MiB/738.7 MiB] 100% Done  22.2 MiB/s ETA 00:00:00           
Operation completed over 1 objects/738.7 MiB.                                    
Archive:  ./SimplerEnv/checkpoints/rt_1_x_tf_trained_for_002272480_step.zip
   creating: ./SimplerEnv/checkpoints/rt_1_x_tf_trained_for_002272480_step/
  inflating: ./SimplerEnv/checkpoints/rt_1_x_tf_trained_for_002272480_step/ckpt-2272480.data-00000-of-00001  
  inflating: ./SimplerEnv/checkpoints/rt_1_x_tf_trained_for_002272480_step/fingerprint.pb  
  inflating: ./SimplerEnv/checkpoints/rt_1_x_tf_trained_for_002272480_step/policy_specs.pbtxt  
  inflating: ./SimplerE

In [ ]:
#@title Run inference

obs, reset_info = env.reset()
instruction = env.get_language_instruction()
model.reset(instruction)
print(instruction)

image = get_image_from_maniskill2_obs_dict(env, obs)  # np.ndarray of shape (H, W, 3), uint8
images = [image]
predicted_terminated, success, truncated = False, False, False
timestep = 0
while not (predicted_terminated or truncated):
    # step the model; "raw_action" is raw model action output; "action" is the processed action to be sent into maniskill env
    raw_action, action = model.step(image)
    predicted_terminated = bool(action["terminate_episode"][0] > 0)
    obs, reward, success, truncated, info = env.step(
        np.concatenate([action["world_vector"], action["rot_axangle"], action["gripper"]])
    )
    print(timestep, info)
    # update image observation
    image = get_image_from_maniskill2_obs_dict(env, obs)
    images.append(image)
    timestep += 1

episode_stats = info.get("episode_stats", {})
print(f"Episode success: {success}")

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_language_instruction to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_language_instruction` for environment variables or `env.get_wrapper_attr('get_language_instruction')` that will search the reminding wrappers.
  logger.warn(


pick coke can


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.robot_uid to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.robot_uid` for environment variables or `env.get_wrapper_attr('robot_uid')` that will search the reminding wrappers.
  logger.warn(


0 {'elapsed_steps': 1, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
1 {'elapsed_steps': 2, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
2 {'elapsed_steps': 3, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
3 {'elapsed_steps': 4, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False)

In [13]:
import numpy as np
from simpler_env.utils.env.observation_utils import get_image_from_maniskill2_obs_dict
import mediapy

# 测试次数
test_episodes = 20

# 统计成功和失败次数
success_count = 0
failure_count = 0

task_name = "google_robot_pick_coke_can"  # 选择任务名称
if 'env' in locals():
    print("Closing existing env")
    env.close()
    del env

env = simpler_env.make(task_name)

# 关闭 denoiser，避免内存问题
sapien.render_config.rt_use_denoiser = False

for episode in range(test_episodes):
    obs, reset_info = env.reset()
    instruction = env.get_language_instruction()
    print(f"Episode {episode + 1} - Instruction: {instruction}")

    model.reset(instruction)
    image = get_image_from_maniskill2_obs_dict(env, obs)  # 获取图像观测
    images = [image]  # 存储每个 episode 的图像序列
    predicted_terminated, success, truncated = False, False, False
    timestep = 0

    while not (predicted_terminated or truncated):
        # 模型推理并生成动作
        raw_action, action = model.step(image)
        predicted_terminated = bool(action["terminate_episode"][0] > 0)
        obs, reward, success, truncated, info = env.step(
            np.concatenate([action["world_vector"], action["rot_axangle"], action["gripper"]])
        )
        print(f"Timestep {timestep}: Info: {info}")

        # 更新图像观测
        image = get_image_from_maniskill2_obs_dict(env, obs)
        images.append(image)
        timestep += 1

    episode_stats = info.get("episode_stats", {})
    print(f"Episode {episode + 1} - Success: {success}")

    # 播放当前 episode 的视频
    print(task_name, model.name if hasattr(model, 'name') else 'Unnamed Model')
    mediapy.show_video(images, fps=10)

    # 根据结果更新统计
    if success:
        success_count += 1
    else:
        failure_count += 1

# 输出测试结果
print(f"\nTask: {task_name}")
print(f"Total Episodes: {test_episodes}")
print(f"Successes: {success_count}")
print(f"Failures: {failure_count}")


Closing existing env


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_language_instruction to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_language_instruction` for environment variables or `env.get_wrapper_attr('get_language_instruction')` that will search the reminding wrappers.
  logger.warn(


Episode 1 - Instruction: pick coke can


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.robot_uid to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.robot_uid` for environment variables or `env.get_wrapper_attr('robot_uid')` that will search the reminding wrappers.
  logger.warn(


Timestep 0: Info: {'elapsed_steps': 1, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 1: Info: {'elapsed_steps': 2, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 2: Info: {'elapsed_steps': 3, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 3: Info: {'elapsed_steps': 4, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats':

...

Episode 19 - Instruction: pick coke can
Timestep 0: Info: {'elapsed_steps': 1, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 1: Info: {'elapsed_steps': 2, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 2: Info: {'elapsed_steps': 3, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 3: Info: {'elapsed_steps': 4, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': F

This browser does not support the video tag.


Episode 20 - Instruction: pick coke can
Timestep 0: Info: {'elapsed_steps': 1, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 1: Info: {'elapsed_steps': 2, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 2: Info: {'elapsed_steps': 3, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
Timestep 3: Info: {'elapsed_steps': 4, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': F

This browser does not support the video tag.



Task: google_robot_pick_coke_can
Total Episodes: 20
Successes: 15
Failures: 5


In [ ]:
print(task_name, model_name)
mediapy.show_video(images, fps=10)

google_robot_pick_coke_can rt_1_x


This browser does not support the video tag.


## Gallery

In [ ]:
# @markdown RT-1-X close drawer
print(task_name, model_name)
mediapy.show_video(images, fps=10)
# Note: we turned off the denoiser as the colab kernel will crash if it's turned on
# To use the denoiser, please git clone our SIMPLER environments
# and perform evaluations locally.

This browser does not support the video tag.


In [ ]:
# @markdown Octo-base widowx_put_eggplant_in_basket
print(task_name, model_name)
mediapy.show_video(images, fps=10)

widowx_put_eggplant_in_basket octo-base


This browser does not support the video tag.
